In [7]:
import einops
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformer_lens import HookedTransformer, HookedTransformerConfig, utils


Generating lists of a fixed length parameter

In [10]:
# data generation
import random

FIXED_LENGTH = 2

def generateLists(n):
    output = []
    for _ in range(n):
        curr = []
        for _ in range(FIXED_LENGTH):
            curr.append(random.randint(0, 100))

        # maximum = max(curr)
        # output.append((curr, maximum))
        output.append(curr)

    output = torch.tensor(output)
    return output

tensor([[30, 47],
        [94, 57],
        [48, 10],
        [12, 49],
        [73, 15]])

Model paramters. We are using one layer, one attention head (which pays to the 
tokens in contex to another), the dimensions of the model, dimension of the head,
vocab is the size of the logits

In [12]:
# model parameter constants
N_LAYERS = 1
N_HEADS = 1
D_MODEL = 32
D_HEAD = 32
D_MLP = None
D_VOCAB = 64
SEED = 123
DEVICE = torch.device("mps")

In [ ]:
# model setup
cfg = HookedTransformerConfig(
    d_model=D_MODEL,
    n_layers=N_LAYERS,
    n_heads=N_HEADS,
    d_head=D_HEAD,
    n_ctx=2,
    d_vocab=D_VOCAB,
    act_fn="relu",
    seed=SEED,
    device=DEVICE,
    attn_only=True
)

# hooked transformer used for interpretation later
model = HookedTransformer(cfg, move_to_device=True)

In [ ]:
def loss_function(logits, tokens, return_per_token=False):
    # we take the last element of the logits to make the next prediction
    logits = logits[:, -1, :]
    true_maximum = torch.max(tokens, dim=1)[0]
    log_probs = logits.log_softmax(-1) 
    
